In [1]:
# !pip3 install statsforecast==1.7.4 neuralforecast==1.6.4 utilsforecast==0.1.1 -q

In [2]:
# !pip3 install statsforecast

In [ ]:
# df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/air-passengers.csv')


In [24]:
import streamlit as st 
st.__version__

'1.38.0'

In [25]:
import plotly as pt 
pt.__version__

'5.24.0'

In [22]:
from neuralforecast.utils import AirPassengersDF

In [23]:
AirPassengersDF.shape 

(144, 3)

In [ ]:
AirPassengersDF.to_csv('AirPassengersDF.csv',index=False)

In [4]:
AirPassengersDF.head()

,unique_id,ds,y
0,1.0,1949-01-31,112.0
1,1.0,1949-02-28,118.0
2,1.0,1949-03-31,132.0
3,1.0,1949-04-30,129.0
4,1.0,1949-05-31,121.0


In [5]:
import statsforecast
statsforecast.__version__

'1.7.4'

In [22]:
def train_statistical_model(df, seasonal_length, date_format, data_frequency, forecast_horizon, 
                             ignore_neg_fcsts, error_metric, eval):
    df = df.copy()
    print('under stats model', df.columns)
    stats = StatsForecastModels(
        seasonal_length=seasonal_length,
        freq=data_frequency,
        date_format=date_format,
        )
    
    stats.fit(df)
    df_pred = stats.predict(horizon=forecast_horizon, ignore_neg_fcsts=ignore_neg_fcsts)

    if ( eval ):
        df_crossval = stats.crossvalidation(error_metric=error_metric)
        df_performance = evaluate_cross_validation(df_crossval, error_metric)
        df_rank = rank_forecast(df_pred[df_pred['unique_id'].isin(df_performance.index)].reset_index(drop=True), df_performance)
        
        # ids_bucket_1 = list(( df.groupby('unique_id')['ds'].count()[ (df.groupby('unique_id')['ds'].count()<=27) & (df.groupby('unique_id')['ds'].count()>=12) ] ).index)

        # if(len(ids_bucket_1)>0):
        #     df_rank_bucket_1 = df_pred[df_pred['unique_id'].isin(ids_bucket_1)][['unique_id','ds','AutoTheta']].reset_index(drop=True)
        #     df_rank_bucket_1.rename(columns={'AutoTheta':'forecast'},inplace=True)
        #     df_rank_bucket_1['model'] = 'AutoTheta'
        #     df_rank_bucket_1['rank'] = 1
            
        #     df_rank = pd.concat([df_rank_bucket_1,df_rank])
    
    else:
        df_performance,df_rank,df_crossval = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    return df_pred, df_performance, df_crossval, df_rank


def evaluate_cross_validation(df, metric):
    error_metric = globals()[metric]
    models = df.drop(columns=["unique_id", "ds", "cutoff", "y"]).columns.tolist()
    evals = []
    for cutoff in df["cutoff"].unique():
        eval_ = evaluate(
            df[df["cutoff"] == cutoff], metrics=[error_metric], models=models
        )
        evals.append(eval_)
    evaluated = pd.concat(evals)
    evaluated = evaluated.groupby("unique_id").mean(numeric_only=True)

    evaluated.insert(0, "best_model", evaluated.idxmin(axis=1))
    return evaluated


def rank_forecast(pred__, evaluate__):

    df_tmp_transform_ = evaluate__.reset_index().T.reset_index()
    headers = df_tmp_transform_.iloc[0]
    df_tmp_rank_ = pd.DataFrame(df_tmp_transform_.values[1:], columns=headers)
    df_tmp_rank_.rename(columns={"unique_id": "models"}, inplace=True)
    df_tmp_rank_ = df_tmp_rank_[df_tmp_rank_["models"] != "best_model"].reset_index(
        drop=True
    )
    df_int_ = pd.DataFrame()
    for id_ in evaluate__.index:
        df_tmp_rank_[id_] = df_tmp_rank_[id_].astype(float)
        df_tmp_ = df_tmp_rank_[["models", id_]].sort_values(by=id_)
        df_tmp_pred = pd.DataFrame()
        rank = 1
        for mdl_ in df_tmp_["models"]:
            df_tmp_pred = pred__[pred__["unique_id"] == id_].reset_index(drop=True)[
                ["unique_id", "ds"]
            ]
            df_tmp_pred["model"] = mdl_
            df_tmp_pred["forecast"] = pred__[mdl_]
            df_tmp_pred["rank"] = rank
            df_int_ = pd.concat([df_int_, df_tmp_pred], ignore_index=True)
            rank += 1
    
    df_int_ = df_int_[~df_int_['forecast'].isna()].reset_index(drop=True)

    return df_int_

# def evaluate_cross_validation(df, metric):
#     df = df.copy()
#     error_metric = globals()[metric]
#     models = df.drop(columns=["unique_id", "ds", "cutoff", "y"]).columns.tolist()
#     evals = []
#     for cutoff in df["cutoff"].unique():
#         eval_ = evaluate(
#             df[df["cutoff"] == cutoff], metrics=[error_metric], models=models
#         )
#         evals.append(eval_)
#     evaluated = pd.concat(evals)
#     evaluated = evaluated.groupby("unique_id").mean(numeric_only=True)

#     evaluated.insert(0, "best_model", evaluated.idxmin(axis=1))
#     return evaluated


In [23]:
def build_forecast_model(df, model_type, date_format, data_frequency, seasonal_length, forecast_horizon, ignore_neg_fcsts, error_metric, eval=True):
    
    if(model_type=='statistical'):
        print('getting resulst for statsitcal model')
        print(df.columns)
        df_pred, df_performance, df_crossval, df_rank = train_statistical_model(df, seasonal_length, date_format, data_frequency, forecast_horizon, ignore_neg_fcsts, error_metric, eval)
    return df_pred, df_performance, df_crossval, df_rank

In [24]:
from statsforecast import StatsForecast
import matplotlib.pyplot as plt
import argparse
from statsforecast.models import (
    AutoARIMA,
    AutoETS,
    AutoCES,
    AutoTheta,
    AutoRegressive,
    SeasonalExponentialSmoothingOptimized,
    Holt,
    HoltWinters,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    SeasonalNaive,
    Naive
)
from utilsforecast.losses import smape, mape, mse, rmse, mae, rmae, mase
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from utilsforecast.losses import smape, mape, mse, rmse, mae, rmae, mase
from utilsforecast.evaluation import evaluate
import pandas as pd
import numpy as np
from pandas import DataFrame
from typing import Optional
import warnings
import cProfile

warnings.filterwarnings("ignore")

from statsforecast.utils import ConformalIntervals

In [25]:


# STATS_MDL = [
#     "AutoARIMA",
#     "AutoETS",
#     "AutoCES",
#     "AutoTheta",
#     "AutoRegressive",
#     "SeasonalExponentialSmoothingOptimized",
#     "Holt",
#     "HoltWinters",
#     "SeasonalNaive",
#     "SeasonalWindowAverage",
#     "RandomWalkWithDrift",
# ]



MDL_PARAMETERS = {
    "AutoARIMA": {"season_length": 1},
    "AutoETS": {"season_length": 1},
    "AutoCES": {"season_length": 1},
    "AutoTheta": {"season_length": 1},
    "AutoRegressive": {"lags": 1,"include_drift":True},
    "SeasonalExponentialSmoothingOptimized": {"season_length": 1},
    "Holt": {"season_length": 1, "alias":"Holt"},
    "HoltWinters": {"season_length": 1, "alias":'HoltWinters'},
    "SeasonalNaive": {"season_length": 1},
    "SeasonalWindowAverage": {"season_length": 1, 'window_size':1}, 
}

# ENSEMBLE_MAPPING = {
#     "stats_ensemble": STATS_MDL
# }

# MDL_SPEED_MAPPING = {
    
#     "all": STATS_MDL,
    
#     "test": STATS_MDL
# }

MDL_RENAME_MAPPING = {
    "RWD": "RandomWalkWithDrift",
    "CES": "AutoCES",
    "SeasWA": "SeasonalWindowAverage",
    "SES": "SimpleExponentialSmoothing",
    "SESOpt": "SimpleExponentialSmoothingOptimized",
    "SeasonalES":"SeasonalExponentialSmoothing",
    "SeasESOpt": "SeasonalExponentialSmoothingOptimized"
}

In [26]:



class StatsForecastModels:
    def __init__(
        self, seasonal_length: int, freq: str, date_format: str
    ) -> None:

        self.models = [
            "AutoARIMA",
            "AutoETS",
            "AutoCES",
            "AutoTheta",
            "AutoRegressive",
            "SeasonalExponentialSmoothingOptimized",
            "Holt",
            "HoltWinters",
            "SeasonalNaive",
            "SeasonalWindowAverage",
            "RandomWalkWithDrift",
        ]

        if( seasonal_length > 1):
            self.models.remove('Holt')
        else:
            self.models.remove('HoltWinters')


#         if len(set(self.models) - set(STATS_MDL)) != 0:
#             mdl_np = ",".join(list(set(model_type) - set(STATS_MDL)))
#             raise Exception(mdl_np, "models are not present in module")

        self.seasonal_length = seasonal_length
        self.freq = freq
        self.date_format = date_format
        self.model_pos = None
        self.ignore_neg_fcsts = False

    def set_mdl_param(self):
        if self.seasonal_length is not None:
            for mdl_name, mdl_params in MDL_PARAMETERS.items():
                if "season_length" in mdl_params.keys():
                    mdl_params["season_length"] = self.seasonal_length
                if "window_size" in mdl_params.keys():
                    window_size = int( (self.train_df.ds.nunique()*0.20) / self.seasonal_length )
                    if(window_size<=1):
                        mdl_params['window_size'] = 1
                    else:
                        mdl_params['window_size'] = window_size
                        
                if "lags" in mdl_params.keys():
                    mdl_params["lags"] = self.seasonal_length
        
    def data_validation(self):
        try:
#             date_mapping = {"MM/DD/YYYY": "%m-%d-%Y", "DD/MM/YYYY": "%d-%m-%Y", "DD-MM-YYYY": "%d-%m-%Y", "MM-DD-YYYY": "%m-%d-%Y", "YYYY/MM/DD": "%Y-%m-%d"}
            date_mapping = {"DD/MM/YYYY": "%d/%m/%Y",
                            "MM/DD/YYYY": "%m/%d/%Y",
                            "YYYY/MM/DD": "%Y/%m/%d",
                            "DD-MM-YYYY": "%d-%m-%Y",
                            "MM-DD-YYYY": "%m-%d-%Y",
                            "YYYY-MM-DD": "%Y-%m-%d",}
    
            self.train_df["ds"] = pd.to_datetime(
                self.train_df["ds"], format=date_mapping[self.date_format]
            )
        except KeyError as e:
            print(f"Error: Invalid date format: {self.date_format}")
            return
        
        self.format_validation()
        return None

    def format_validation(self):
        return None

    def fit(self, train_df: pd.DataFrame):
        self.train_df = train_df
        
        if((self.freq=='M') | (self.freq=='MS')):
            self.ids_bucket_1 = list(( train_df.groupby('unique_id')['ds'].count()[(train_df.groupby('unique_id')['ds'].count()<=27)] ).index)
            self.ids_bucket_2 = list(( train_df.groupby('unique_id')['ds'].count()[(train_df.groupby('unique_id')['ds'].count()<=35) & (train_df.groupby('unique_id')['ds'].count()>=28)] ).index)
            self.ids_bucket_3 = list(( train_df.groupby('unique_id')['ds'].count()[(train_df.groupby('unique_id')['ds'].count()>=36)] ).index)

        if(self.freq=='W'):
            self.ids_bucket_1 = list(( train_df.groupby('unique_id')['ds'].count()[(train_df.groupby('unique_id')['ds'].count()<=12) & (train_df.groupby('unique_id')['ds'].count()>=4)] ).index)
            self.ids_bucket_2 = list(( train_df.groupby('unique_id')['ds'].count()[ (train_df.groupby('unique_id')['ds'].count()>=13) ] ).index)
            self.ids_bucket_3 = list([])
            
        
        self.set_mdl_param()
        
        try:
            self.data_validation()
            selected_models = []
            for model_name in self.models:
                if model_name in MDL_PARAMETERS.keys():
                    selected_models.append(
                        globals()[model_name](**MDL_PARAMETERS[model_name])
                    )
                else:
                    selected_models.append(globals()[model_name]())
                                        
            sf_naive = StatsForecast( 
                models=[Naive()],
                freq=self.freq
                )

            self.sf_naive = sf_naive.fit(self.train_df)

            if(len(self.ids_bucket_1)>0):
                
                sf = StatsForecast(
                models=selected_models,
                freq=self.freq,
                fallback_model=Naive()
                )
                self.sf_bucket_1 = sf.fit(self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_1)].reset_index(drop=True))
                
            if(len(self.ids_bucket_2)>0):
            
                sf = StatsForecast(
                models=selected_models,
                freq=self.freq,
                fallback_model=Naive()
                )
                self.sf_bucket_2 =sf.fit(self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_2)].reset_index(drop=True))
            
            if(len(self.ids_bucket_3)>0):
                
                sf = StatsForecast(
                models=selected_models,
                freq=self.freq,
                fallback_model=Naive()
                )
                self.sf_bucket_3 = sf.fit(self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_3)].reset_index(drop=True))

                            
            return self
        
        except Exception as e:
            print(f"Error during fitting: {e}")
            return None

    def predict(self, horizon: int,ignore_neg_fcsts = False):
        
        self.horizon = horizon
        self.ignore_neg_fcsts = ignore_neg_fcsts
        
        self.pred__ = pd.DataFrame()
        
        try:
            
            if(len(self.ids_bucket_1)>0):
                self.pred_bucket_1 = self.sf_bucket_1.predict(horizon).rename(columns=MDL_RENAME_MAPPING) 
                
                mdl_rename = list(set(self.models) - set(self.pred_bucket_1.columns))
                
                if(len(mdl_rename)==1):
                    self.pred_bucket_1 = self.pred_bucket_1.rename(columns={'Naive':mdl_rename[0]})
                                        
                self.pred__ = pd.concat([self.pred__,self.pred_bucket_1])
    
                
            if(len(self.ids_bucket_2)>0):
                self.pred_bucket_2 = self.sf_bucket_2.predict(horizon).rename(columns=MDL_RENAME_MAPPING) 
                
                mdl_rename = list(set(self.models) - set(self.pred_bucket_2.columns))
                
                if(len(mdl_rename)==1):
                    self.pred_bucket_2 = self.pred_bucket_2.rename(columns={'Naive':mdl_rename[0]})
                        
                self.pred__ = pd.concat([self.pred__,self.pred_bucket_2])
            
                        
            if(len(self.ids_bucket_3)>0):
                self.pred_bucket_3 = self.sf_bucket_3.predict(horizon).rename(columns=MDL_RENAME_MAPPING)
                
                mdl_rename = list(set(self.models) - set(self.pred_bucket_3.columns))
                
                if(len(mdl_rename)==1):
                    self.pred_bucket_3 = self.pred_bucket_3.rename(columns={'Naive':mdl_rename[0]})
                        
                self.pred__ = pd.concat([self.pred__,self.pred_bucket_3])
            

            self.pred_naive = self.sf_naive.predict(horizon)

            self.pred__ = self.pred__.merge(self.pred_naive,on=['unique_id','ds'],how='left').reset_index()


            # model_columns = self.pred__.columns[2:-1].values
            
            def replace_with_nan(group):
                for model in self.models:
                    if( (group[model]!=group['Naive']).sum() == 0 ):
                        group[model] = np.nan
                return group

            self.pred__ = self.pred__.groupby('unique_id').apply(replace_with_nan).reset_index(drop=True)

            
        except:
            raise Exception("Model not fitted yet. Call fit method first.")
            
        self.pred__['stats_ensemble'] = self.pred__[self.models].mean(axis=1).drop(columns='Naive')

        self.model_pos = list((self.pred__[self.pred__.iloc[:,2:].columns] >= 0).all().index)

        if(ignore_neg_fcsts == True):
            self.pred__= self.pred__[['unique_id','ds'] + self.model_pos]

        return self.pred__
    

    def crossvalidation(
        self, error_metric: str, h: Optional[int] = None
    ):
        
        crossvalidation_df = pd.DataFrame()

        def replace_with_nan(group):
                for model in self.models:
                    if( (group[model]!=group['Naive']).sum() == 0 ):
                        group[model] = np.nan
                return group
            
        if(len(self.ids_bucket_2)>0):
            
            df_f = self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_2)].reset_index(drop=True)
            
            if((self.freq=='M') | (self.freq=='MS')):
                h = df_f.groupby('unique_id')['ds'].count().min() - 25
            if(self.freq=='W'):
                h = 4

            sf_naive_ = StatsForecast( 
                models=[Naive()],
                freq=self.freq,
                verbose=True)
            
            df_cross_naive = sf_naive_.cross_validation(df=self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_2)].reset_index(drop=True) , h=h).reset_index()

            crossvalidation_df_bucket_2 = self.sf_bucket_2.cross_validation(h=h)

            crossvalidation_df_bucket_2 = crossvalidation_df_bucket_2.reset_index().rename(
                columns=MDL_RENAME_MAPPING
            )

            crossvalidation_df_bucket_2 = crossvalidation_df_bucket_2.merge(df_cross_naive[['unique_id','ds','Naive']],on=['unique_id','ds'],how='left')

            crossvalidation_df_bucket_2 = crossvalidation_df_bucket_2.groupby('unique_id').apply(replace_with_nan).reset_index(drop=True)
            
            crossvalidation_df = pd.concat([crossvalidation_df,crossvalidation_df_bucket_2])
            
        if(len(self.ids_bucket_3)>0):

            if((self.freq=='M') | (self.freq=='MS')): 
                h = 12
            if(self.freq=='W'): 
                h = 4

            sf_naive_ = StatsForecast( 
                models=[Naive()],
                freq=self.freq,
                verbose=True)

            df_cross_naive = sf_naive_.cross_validation(df=self.train_df[self.train_df['unique_id'].isin(self.ids_bucket_3)].reset_index(drop=True) ,h=h).reset_index()

            crossvalidation_df_bucket_3 = self.sf_bucket_3.cross_validation(h=h)

            crossvalidation_df_bucket_3 = crossvalidation_df_bucket_3.reset_index().rename(
                columns=MDL_RENAME_MAPPING
            )

            crossvalidation_df_bucket_3 = crossvalidation_df_bucket_3.merge(df_cross_naive[['unique_id','ds','Naive']],on=['unique_id','ds'],how='left')

            crossvalidation_df_bucket_3 = crossvalidation_df_bucket_3.groupby('unique_id').apply(replace_with_nan).reset_index(drop=True)
            
            crossvalidation_df = pd.concat([crossvalidation_df,crossvalidation_df_bucket_3])

        if( (len(self.ids_bucket_3)>0) | (len(self.ids_bucket_2)>0) ):
            
            crossvalidation_df['stats_ensemble'] = crossvalidation_df[self.models].mean(axis=1)

            if(self.ignore_neg_fcsts):
                crossvalidation_df = crossvalidation_df[['unique_id','ds','cutoff','y']+self.model_pos]

        return crossvalidation_df

In [27]:
df_pred, df_performance, df_crossval, df_rank = build_forecast_model(df=AirPassengersDF, model_type='statistical', date_format='YYYY-MM-DD', data_frequency='M', seasonal_length=12, forecast_horizon=16, ignore_neg_fcsts=False, error_metric='rmse')

getting resulst for statsitcal model
Index(['unique_id', 'ds', 'y'], dtype='object')
under stats model Index(['unique_id', 'ds', 'y'], dtype='object')


Cross Validation Time Series 1: 100%|██████████| 1/1 [00:00<00:00, 14979.66it/s]


In [29]:
df_pred.head()

,unique_id,ds,AutoARIMA,AutoETS,AutoCES,AutoTheta,AutoRegressive,SeasonalExponentialSmoothingOptimized,HoltWinters,SeasonalNaive,SeasonalWindowAverage,RandomWalkWithDrift,Naive,stats_ensemble
0,1.0,1961-01-31,447.836945,442.357178,453.034180,442.940796,460.672668,416.427979,453.088745,417.0,388.5,434.237762,432.0,435.609619
1,1.0,1961-02-28,423.679565,428.267365,429.340393,432.229370,423.261719,390.507568,427.476532,391.0,366.5,436.475525,432.0,414.873779
2,1.0,1961-03-31,453.336334,492.974792,488.644714,495.306091,445.753448,418.865601,460.631775,419.0,412.5,438.713287,432.0,452.572601
3,1.0,1961-04-30,496.538574,477.369995,500.289551,482.306244,472.159729,460.345215,497.036804,461.0,428.5,440.951050,432.0,471.649719
4,1.0,1961-05-31,508.468170,477.602814,519.799622,487.493134,507.811127,471.474304,507.507538,472.0,446.0,443.188812,432.0,484.134583


In [30]:
df_performance.head()

,best_model,AutoARIMA,AutoETS,AutoCES,AutoTheta,AutoRegressive,SeasonalExponentialSmoothingOptimized,HoltWinters,SeasonalNaive,SeasonalWindowAverage,RandomWalkWithDrift,Naive,stats_ensemble
unique_id,,,,,,,,,,,,,
1.0,AutoCES,23.955328,40.083622,14.657907,24.985178,33.507935,51.147297,26.228498,50.708317,73.446129,92.666359,102.97654,34.97551


In [31]:
df_crossval.head()

,unique_id,ds,cutoff,y,AutoARIMA,AutoETS,AutoCES,AutoTheta,AutoRegressive,SeasonalExponentialSmoothingOptimized,HoltWinters,SeasonalNaive,SeasonalWindowAverage,RandomWalkWithDrift,Naive,stats_ensemble
0,1.0,1960-01-31,1959-12-31,417.0,424.160156,406.651276,410.827393,412.711884,393.031372,359.797455,409.680145,360.0,350.0,407.236633,405.0,393.409607
1,1.0,1960-02-29,1959-12-31,391.0,407.081696,401.732910,391.996124,404.640472,374.763031,341.758270,390.044067,342.0,330.0,409.473297,405.0,379.348999
2,1.0,1960-03-31,1959-12-31,419.0,470.860535,456.289642,456.414459,466.810883,408.308929,405.559357,448.929810,406.0,384.0,411.709930,405.0,431.488342
3,1.0,1960-04-30,1959-12-31,461.0,460.913605,440.870514,447.666779,449.600372,403.529755,395.519958,435.445007,396.0,372.0,413.946564,405.0,421.549255
4,1.0,1960-05-31,1959-12-31,472.0,484.900879,440.333923,472.150970,454.047760,444.491333,419.429169,454.699951,420.0,391.5,416.183197,405.0,439.773743


In [32]:
df_rank.head()

,unique_id,ds,model,forecast,rank
0,1.0,1961-01-31,AutoCES,453.034180,1
1,1.0,1961-02-28,AutoCES,429.340393,1
2,1.0,1961-03-31,AutoCES,488.644714,1
3,1.0,1961-04-30,AutoCES,500.289551,1
4,1.0,1961-05-31,AutoCES,519.799622,1


In [34]:
# df_rank

In [35]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from io import BytesIO
# Import your forecasting model libraries here, e.g., Prophet, ARIMA


In [36]:
!streamlit run app.py 


      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  ^C
2024-11-08 14:17:28.600 


In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('air-passengers.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   unique_id  144 non-null    object
 1   ds         144 non-null    object
 2   y          144 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.5+ KB


In [5]:
df.head()

,unique_id,ds,y
0,AirPassengers,1949-01-01,112
1,AirPassengers,1949-02-01,118
2,AirPassengers,1949-03-01,132
3,AirPassengers,1949-04-01,129
4,AirPassengers,1949-05-01,121


In [8]:
df.describe()

,ds,y
count,144,144.000000
mean,1954-12-16 05:00:00,280.298611
min,1949-01-01 00:00:00,104.000000
25%,1951-12-24 06:00:00,180.000000
50%,1954-12-16 12:00:00,265.500000
75%,1957-12-08 18:00:00,360.500000
max,1960-12-01 00:00:00,622.000000
std,NaN,119.966317


In [9]:
next_dates = pd.date_range(df['ds'].iloc[-1], periods=13, freq='MS')[1:]
next_df = pd.DataFrame({
    'unique_id': ['AirPassengers'] * 12,
    'ds': next_dates,
    'y': [None] * 12  # No values for 'y' yet
})
print(next_df)

        unique_id         ds     y
0   AirPassengers 1961-01-01  None
1   AirPassengers 1961-02-01  None
2   AirPassengers 1961-03-01  None
3   AirPassengers 1961-04-01  None
4   AirPassengers 1961-05-01  None
5   AirPassengers 1961-06-01  None
6   AirPassengers 1961-07-01  None
7   AirPassengers 1961-08-01  None
8   AirPassengers 1961-09-01  None
9   AirPassengers 1961-10-01  None
10  AirPassengers 1961-11-01  None
11  AirPassengers 1961-12-01  None


In [12]:
import numpy as np 
np.random.randn(12).tolist()

[-0.16358273904128498,
 -0.0654256827131722,
 2.15631965714235,
 0.15671439031900888,
 0.42495577088873454,
 -1.0603430872777562,
 0.23324347105152354,
 1.010772711438264,
 -0.9020784786230626,
 1.04290777123561,
 1.7493107172722036,
 -2.1962222564138796]

In [19]:
models = ["Model_ARIMA", "Model_Prophet", "Model_LSTM"]  # Replace with actual model names
forecast_horizon = 12 

forecast_results = pd.DataFrame()
forecast_results[f'ds'] = pd.date_range(df['ds'].iloc[-1], periods=forecast_horizon+1, freq='MS')[1:]
forecast_results[f'unique_id']= df['unique_id'][:forecast_horizon]
for model in models:
    forecast_results[model] = np.random.randn(forecast_horizon).tolist()

In [20]:
df.tail()

,unique_id,ds,y
139,AirPassengers,1960-08-01,606
140,AirPassengers,1960-09-01,508
141,AirPassengers,1960-10-01,461
142,AirPassengers,1960-11-01,390
143,AirPassengers,1960-12-01,432


In [21]:
forecast_results

,ds,unique_id,Model_ARIMA,Model_Prophet,Model_LSTM
0,1961-01-01,AirPassengers,1.553429,1.158864,-0.338632
1,1961-02-01,AirPassengers,-0.128835,0.456952,0.086386
2,1961-03-01,AirPassengers,0.709753,-0.024270,-0.181842
3,1961-04-01,AirPassengers,-1.450626,0.769838,0.486375
4,1961-05-01,AirPassengers,-1.232098,0.235793,0.242796
5,1961-06-01,AirPassengers,-0.892027,-0.108596,-1.186534
6,1961-07-01,AirPassengers,0.222959,-0.625139,-0.958414
7,1961-08-01,AirPassengers,1.907500,-0.740632,-0.060123
8,1961-09-01,AirPassengers,1.068454,0.103857,0.264537
9,1961-10-01,AirPassengers,-0.022451,0.693684,-0.176851
